# Load and preprocess 2012 data

We will, over time, look over other years. Our current goal is to explore the features of a single year.

---

In [2]:
%pylab --no-import-all inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


## Load the data.

---

If this fails, be sure that you've saved your own data in the prescribed location, then retry.

In [15]:
file = "../data/interim/2012data.dta"
df_rawest = pd.read_stata(file)

In [16]:
good_columns = [#'campfin_limcorp', # "Should gov be able to limit corporate contributions"
    'pid_x',  # Your own party identification
    
    'abortpre_4point',  # Abortion
    'trad_adjust',  # Moral Relativism
    'trad_lifestyle',  # "Newer" lifetyles
    'trad_tolerant',  # Moral tolerance
    'trad_famval',  # Traditional Families
    'gayrt_discstd_x',  # Gay Job Discrimination
    'gayrt_milstd_x',  # Gay Military Service
    
    'inspre_self',  # National health insurance
    'guarpr_self',  # Guaranteed Job
    'spsrvpr_ssself',  # Services/Spending
    
    'aa_work_x',  # Affirmative Action  ( Should this be aapost_hire_x? )   
]

df_raw = df_rawest[good_columns]

## Clean the data
---

In [21]:
def convert_to_int(s):
    """Turn ANES data entry into an integer.
    
    >>> convert_to_int("1. Govt should provide many fewer services")
    1
    >>> convert_to_int("2")
    2
    """
    try:
        return int(s.partition('.')[0])
    except ValueError:
        warnings.warn("Couldn't convert: "+s)
        return np.nan
    except AttributeError:
        return s

def negative_to_nan(value):
    """Convert negative values to missing.
    
    ANES codes various non-answers as negative numbers.
    For instance, if a question does not pertain to the 
    respondent.
    """
    return value if value >= 0 else np.nan

def lib1_cons2_neutral3(x):
    """Rearrange questions where 3 is neutral."""
    return -3 + x if x != 1 else x

def liblow_conshigh(x):
    """Reorder questions where the liberal response is low."""
    return -x

def dem_edu_special_treatment(x):
    """Eliminate negative numbers and {95. Other}"""
    return np.nan if x == 95 or x <0 else x

df = df_raw.applymap(convert_to_int)
df = df.applymap(negative_to_nan)

df.pid_x = df.pid_x.apply(lambda x: -x)
df.abortpre_4point = df.abortpre_4point.apply(lambda x: np.nan if x not in {1, 2, 3, 4} else x)

df.loc[:, 'trad_adjust'] = df.trad_adjust.apply(lambda x: -x)  # 1: moral relativism, 5: no relativism
df.loc[:, 'trad_tolerant'] = df.trad_tolerant.apply(lambda x: -x)  # Tolerance. 1: tolerance, 7: not
df.loc[:, 'gayrt_discstd_x'] = df.gayrt_discstd_x.apply(lambda x: -x)  # protect gays against discrimination
df.loc[:, 'gayrt_milstd_x'] = df.gayrt_milstd_x.apply(lambda x: -x)  # support gay adoption


df.loc[:, 'inspre_self'] = df.inspre_self.apply(lambda x: -x)  # Gov't insurance?
df.loc[:, 'guarpr_self'] = df.guarpr_self.apply(lambda x: -x)  # Gov't guaranteed job?

df.loc[:, 'aa_work_x'] = df.aa_work_x.apply(lambda x: -x)  # affirmative action
# df.loc[:, 'V045194'] = df.V045194.apply(lambda x: -x)  # Racial support
# df.loc[:, 'V045195'] = df.V045195.apply(lambda x: -x)  # Systemic factors?

df.rename(inplace=True, columns=dict(zip(
    good_columns,
    ["PartyID",
    
    "Abortion",
    "MoralRelativism",
    "NewerLifestyles",
    "MoralTolerance",
    "TraditionalFamilies",
    "GayJobDiscrimination",
    "GayMilitaryService",

    "NationalHealthInsurance",
    "StandardOfLiving",
    "ServicesVsSpending",

    "AffirmativeAction",
    "RacialResentment1",
    "RacialResentment2",
    "RacialResentment3",
    "RacialResentment4",
    ]
)))

In [22]:
print("Variables now available: df")

Variables now available: df


In [23]:
df_rawest.pid_x.value_counts()

1. Strong Democrat               1485
2. Not very strong Democract      871
4. Independent                    792
7. Strong Republican              762
3. Independent-Democrat           747
6. Not very strong Republican     623
5. Independent-Republican         610
-2. Missing                        24
Name: pid_x, dtype: int64

In [25]:
df.PartyID.value_counts()

-1.0    1485
-2.0     871
-4.0     792
-7.0     762
-3.0     747
-6.0     623
-5.0     610
Name: PartyID, dtype: int64

In [26]:
df.head()

,PartyID,Abortion,MoralRelativism,NewerLifestyles,MoralTolerance,TraditionalFamilies,GayJobDiscrimination,GayMilitaryService,NationalHealthInsurance,StandardOfLiving,ServicesVsSpending,AffirmativeAction
0,-1.0,2.0,NaN,NaN,NaN,NaN,-1.0,-1.0,-5.0,NaN,5.0,-4.0
1,-1.0,3.0,-1.0,5.0,-1.0,4.0,NaN,NaN,-1.0,-1.0,5.0,-1.0
2,-1.0,4.0,-2.0,4.0,-1.0,1.0,NaN,NaN,-2.0,-2.0,6.0,-2.0
3,-1.0,4.0,-5.0,1.0,-2.0,1.0,-1.0,-1.0,-1.0,-4.0,NaN,-4.0
4,-3.0,4.0,-1.0,4.0,-2.0,4.0,NaN,NaN,-1.0,-3.0,5.0,-2.0


In [27]:
df.to_csv("../data/processed/2012.csv")